In [1]:
## This working simulation function is able to read the location and travel data given the preferences 
## of the traveller and optimise the trip (shorter time but higher utility scores)

In [2]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import math
from geopy.distance import vincenty
import urllib2, requests
import pandas as pd
from tqdm import tqdm
import cPickle as pickle
import sys
sys.setrecursionlimit(10000)
from tsp_solver.greedy import solve_tsp
import matplotlib.pyplot as plt
import seaborn as sns

import googlemaps
%matplotlib inline

In [3]:
fn = '../../../../Browser/Documents/mapapikey.txt'
api = pd.read_csv(fn)
api = api.columns[0]
gmaps = googlemaps.Client(key=api)

In [4]:
import mechanize
import cookielib

# Browser
br = mechanize.Browser()

# Cookie Jar
cj = cookielib.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)

# Follows refresh 0 but not hangs on refresh > 0
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

# Want debugging messages?
#br.set_debug_http(True)
#br.set_debug_redirects(True)
#br.set_debug_responses(True)

# User-Agent setup
# br.addheaders = [('User-agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 5.1; rv:14.0) Gecko/20100101 Firefox/14.0.1',
          'Referer': 'http://www.google.com'}
br.addheaders = [header]

/Users/msor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: gzip transfer encoding is experimental!


In [5]:
# pickle save data
def save_data(db, filename):
    with open('assets/datasets/{}.p'.format(filename), 'wb') as fp:
        pickle.dump(db, fp)

In [6]:
# # pickle load data
def load_data(filename):
    with open('assets/datasets/{}.p'.format(filename), 'rb') as fp:
        read_content = pickle.load(fp)
        return read_content

In [7]:
# pickle save big data
def save_big_data(db, filename):
    with open('../../../../Browser/BigDataSets/{}.p'.format(filename), 'wb') as fp:
        pickle.dump(db, fp)

In [8]:
# # pickle load big data
def load_big_data(filename):
    with open('../../../../Browser/BigDataSets/{}.p'.format(filename), 'rb') as fp:
        read_content = pickle.load(fp)
        return read_content

In [9]:
edges = load_data('edges')

In [10]:
locations = load_data('attraction_locations')
location_details = load_data('location_details')

In [11]:
# get lat long from google maps api
def get_coordinates(attractions, add_suffix='Hong Kong'):
    attractions = attractions.encode('utf-8')
    query = attractions + ', ' + add_suffix
    geocode_result = gmaps.geocode(query)
    if geocode_result == []:
        return None,None,None,None
    else:
        latitude = geocode_result[0]['geometry']['location']['lat']
        longitude = geocode_result[0]['geometry']['location']['lng']
        return attractions, latitude, longitude, geocode_result

In [12]:
# attractions mapping to map the attractions to the geo location details
def location_mapping(attractions):
    locations = load_data('attraction_locations')
    location_details = load_data('location_details')
    for i in tqdm(attractions):
        if i not in locations.keys():
            loc, lat, lon, loc_details = get_coordinates(i.decode(encoding='UTF-8'))
            if loc != None:
                locations[loc] = (lat, lon)
                location_details[loc] = loc_details
    return locations, location_details

In [13]:
# convert location dictionary to location dataframe
def loc_dict2df(locations):
    df = []
    for i, j in locations.items():
        df.append([i, j[0],j[1]])
    df = pd.DataFrame(df, columns=['Location', 'Latitude', 'Longitude'])
    return df

In [14]:
# clear_places = []
# for i in location_details:
#     clear_places.append([i, location_details[i][0]['formatted_address']])
# clear_places = pd.DataFrame(clear_places, columns=['Location','Address'])

In [15]:
# attractions = list(clear_places[clear_places.Address != "Hong Kong"]['Location'])

In [16]:
# this code will export the locations and lat long coordinates for plotting on tableau

# location_coord = loc_dict2df(locations)
# location_coord.to_csv('assets/datasets/location_lat_lon.csv')

In [17]:
# pair up 'nodes' (locations) to create 'edges' (routes) of the graphs
def node_pairing(locations):
    location_pair = []
    for i in range(len(locations)):
        for j in range(i+1,len(locations)):
            location_pair.append( [ {locations.keys()[i] : locations[locations.keys()[i]]}, {locations.keys()[j] : locations[locations.keys()[j]]} ] )
            location_pair.append( [ {locations.keys()[j] : locations[locations.keys()[j]]}, {locations.keys()[i] : locations[locations.keys()[i]]} ] )
    return location_pair
location_pair = node_pairing(locations)

In [18]:
# get the suggested route from the government webpage given lat lon...
def get_route(start_lat, start_lon, end_lat, end_lon):
    latlong = 'http://m.hketransport.gov.hk/getRouteSearchResult.php?ctl26=&ctl30=&ctl36=&ctl40=&ctl41=&ctl42=&slat={}&slon={}&elat={}&elon={}&DDL_BUF_O=400&DDL_BUF_D=400&RB_MODE=RB_MODE1&Btn_RS=Route%20Search&lang=0'.format(start_lat, start_lon, end_lat, end_lon)
    r = br.open(latlong)
    html = r.read()
    return BeautifulSoup(html, "lxml")

In [19]:
def route_finder_hk(location_pair_index):
    i = location_pair_index
    start_lat = i[0][i[0].keys()[0]][0]
    start_lon = i[0][i[0].keys()[0]][1]
    end_lat = i[1][i[1].keys()[0]][0]
    end_lon = i[1][i[1].keys()[0]][1]
    return start_lat, start_lon, end_lat, end_lon

In [20]:
# # save the datasets
# save_data(htmls, 'html_pages')
# save_data(locations, 'attraction_locations')
# save_data(location_details, 'location_details')

In [21]:
# convert direction result into list from page 
def parse_table(table):
    """ Get data from table """
    tmp = [
        [cell.get_text().strip() for cell in row.find_all(['th', 'td'])]
           for row in table.find_all('tr')
    ]
    result = []
    for i in tmp:
        if i != [] and i != ['Route Search'] and i != ['Route Info'] and i != ['Other Info'] and i != ['']:
            if len(i) <= 2:
                result.append(i)
    return result

In [22]:
# convert direction list into dict from list
def build_dict(tbl_list):
    loc_dict = {}
    cross = []
    for i in range(len(tbl_list)):
        if tbl_list[i] != [] :
            if 'Choice ' in tbl_list[i][0][:7]:
                cross.append(i)
    cross.append(len(tbl_list))
    
    for i in range(len(cross)-1):
        loc_dict[tbl_list[cross[i]][0]] = tbl_list[cross[i]:cross[i+1]]
    return loc_dict

In [23]:
# convert direction dict into pd
def build_table(loc_dict):
    loc_table = []
    for i, j in loc_dict.items():
        choice = int(i[7:])
        cost = j[-1:][0][0]
        time = j[-1:][0][1]
        loc_table.append( [choice, cost, time] )
    loc_table = pd.DataFrame(loc_table, columns=['choice', 'cost', 'time'])
    loc_table.sort_values('choice',inplace=True)
    loc_table.index = range(len(loc_table))
    return loc_table

In [24]:
# route details
def route_builder(location_pair, saver = 50, renew=False):
    if renew:
        edges = {}
    else:
        edges = load_data('edges')
    counter = 0
    for i in tqdm(range(len(location_pair))):
        tmp = {}
        a = location_pair[i][0][location_pair[i][0].keys()[0]]
        b = location_pair[i][1][location_pair[i][1].keys()[0]]
        keys = [a,b]
        if str(keys) not in edges.keys():
            start_lat, start_lon, end_lat, end_lon = route_finder_hk(location_pair[i])
            table = get_route(start_lat, start_lon, end_lat, end_lon)
            tbl_list = parse_table(table) # get list of table items from html
            loc_dict = build_dict(tbl_list) # convert the list of choices into logical dictionary
            df = build_table(loc_dict) # build the choices summary into a dataframe
            tmp['detail'] = loc_dict # build detail to the dict
            tmp['result'] = df # build summary to the dict
            tmp['from'] = location_pair[i][0].keys()[0]
            tmp['to'] = location_pair[i][1].keys()[0]
            edges[str(keys)] = tmp
            
            counter = counter + 1
            if counter == saver :
                save_data(edges, 'edges')
                counter = 0
    return edges

In [25]:
def find_distance_approx(start_lat, start_lon, end_lat, end_lon):
    dist = vincenty((start_lat,start_lon), (end_lat, end_lon)).km
    return dist # in kilometers

In [26]:
# edges = route_builder(location_pair)

In [27]:
# save_data(edges, 'edges')

In [28]:
def check_connections(attraction):
    check_k = []
    for k in edges.keys():
        if str(locations[attraction]) in k:
            check_k.append(k)
    connections = []
    for i in check_k:
        connections.append('from: ' + edges[i]['from'])
        connections.append('to: ' + edges[i]['to'])
        connections.append(i)
        if edges[i]['result'].shape == (0,3):
            print edges[i]
            start_lat = locations[edges[i]['from']][0]
            start_lon = locations[edges[i]['from']][1]
            end_lat = locations[edges[i]['to']][0]
            end_lon = locations[edges[i]['to']][1]
            dist = find_distance_approx(start_lat, start_lon, end_lat, end_lon)
            connections.append('Walk ' + str(dist)+'km for '+str(dist*15*1.1) + ' minutes')
        else:
            connections.append(edges[i]['result'])
        connections.append('------')
    return connections

In [29]:
# probability simulation to choose next step based on given scores
def sim_score(scores):
    rand_score = [i / float(np.sum(scores)) for i in scores]
    rand_score = np.cumsum(rand_score)
    num_rand = np.random.rand()
    for i in range(len(scores)):
        if i == 0:
            start = 0
        else:
            start = rand_score[i-1]
        end = rand_score[i]
        if start < num_rand and end >= num_rand:
            score = i
    return score

In [30]:
# convert attraction dataframe to dictionary for easy reference
def change_df2dict(attractions):
    a = attractions[attractions.Duration != 'Hotel']
    a = a[a.Duration != 'Airport']
    a_dict = {}
    a_loc = list(a.Locations)
    for i in range(len(a_loc)):
        tmp = {}
        tmp['Duration'] = a.Duration[i]
        tmp['Score'] = a.Score[i]
        a_dict[a_loc[i]] = tmp
    return a_dict

In [31]:
# find the travel time and fun factor trade-off for given start point and other locations
def get_attraction_stepscore(start, places_remaining, locations, attraction_dict, edges):
    scores_list = []
    for i in places_remaining:
        start_coord = locations[start]
        to_coord = locations[i]
        key = str([start_coord, to_coord])
        try:
            visit = float(edges[key]['result']['time'][0].replace(' minutes',''))
        except:
            visit = float(edges[key]['result']['time'][0])
        time_spent = attraction_dict[i]['Duration'] + visit
        utility_gain = attraction_dict[i]['Score']
        score =  utility_gain / float(time_spent)
        scores_list.append(score)
    return scores_list

In [32]:
def reformat_edges(edges):
    for i in range(len(edges)):
        tmp = edges.keys()[i]
        tmp_edge = edges[tmp]['result']
        try:
            if tmp_edge.shape == (0,3):
                start_lat = locations[edges[tmp]['from']][0]
                start_lon = locations[edges[tmp]['from']][1]
                end_lat = locations[edges[tmp]['to']][0]
                end_lon = locations[edges[tmp]['to']][1]

                dist = find_distance_approx(start_lat, start_lon, end_lat, end_lon)
                tmp_df = [1,0,math.ceil(dist*15*1.1)]
                tmp_df = pd.DataFrame([tmp_df], columns=tmp_edge.columns)
                edges[tmp]['result'] = tmp_df
        except:
            pass
    return edges
# edges = reformat_edges(edges)

In [33]:
# save_data(edges, 'edges_all')

In [34]:
def find_duration(path, hotel, locations, attraction_dict, edges):
    time_spent = 0
    days = 0
    utility = 0
    for i in range(len(path)-1):
        if path[i] == hotel:
            days = days + 1
            
        start_coord = locations[path[i]]
        to_coord = locations[path[i+1]]
        key = str([start_coord,to_coord])
        try:
            stay_time = attraction_dict[path[i+1]]['Duration']
        except:
            stay_time = 0

        try:
            score = attraction_dict[path[i+1]]['Score']
        except:
            score = 0
        try:
            travel_time = edges[key]['result']['time'][0]
            try:
                travel_time = float(edges[key]['result']['time'][0].replace(' minutes',''))
            except:
                pass
        except:
            travel_time = 0
        time_spent = time_spent + travel_time + stay_time
        utility = utility + score

    return time_spent, days, utility

In [35]:
def attractions_visited(path, attraction_dict):
    attraction_list = attraction_dict.keys()
    counter = 0
    for i in attraction_list:
        if i in path:
            counter = counter + 1
    return counter

In [36]:
def find_day_trip(hotel, places_remaining, locations, attraction_dict, edges, day_limit, w_counter):
    not_end_of_day = True
    not_end_of_trip = True
    day_path = [hotel]
    counter = 0
    
    while not_end_of_day and not_end_of_trip:
        # get starting point
        start = day_path[-1]
        
        # find the time_spent to utility ratios
        scores = get_attraction_stepscore(start, places_remaining, locations, attraction_dict, edges)
        
        
        #choose the next step based on the ratio, the higher the ratio the more likely will be selected
        num = sim_score(scores)
        next_visit = places_remaining[num]

        # check if the next visit is valid
        day_path.append(next_visit)
        travelled, day, utility = find_duration(day_path, hotel, locations, attraction_dict, edges)
        
        # check if end of day
        if travelled >= day_limit:
            day_path.remove(next_visit)
            day_path.append(hotel)
            not_end_of_day = False
            travelled2, day2, utility2 = find_duration(day_path, hotel, locations, attraction_dict, edges)
        else:
            places_remaining.remove(next_visit)
            not_end_of_day = True
        
        if places_remaining == []:
            not_end_of_trip = False
        
        
        #### 
        counter = counter + 1
        if counter > w_counter:
            not_end_of_day = False
        ####
    return day_path

In [55]:
def simulation(attractions, stay = None, 
               locations = locations, edges = edges , 
               runs = 100, w_counter = 100, day_length = 12):
    if stay == None:
        stay = float('inf')
    paths = []
    attraction_dict = change_df2dict(attractions)
    hotel = list(attractions[attractions.Duration == 'Hotel'].Locations)[0]
    durations = []
    days = []
    utilities = []
    day_limit = float(day_length) * 60
    keep_run = []
    
    for run in tqdm(range(runs)):
        places_remaining = list(attraction_dict.keys())
        route = []
        keep_running = True
        counter = 0
        
        while keep_running: # not the end of a trip
            
            day_path = find_day_trip(hotel, places_remaining, locations, attraction_dict, edges, day_limit, w_counter)
            
            # check if day has ended
            for row in day_path:
                route.append(row)
                try:
                    places_remaining.remove(row)
                except:
                    pass
            
            # this is the trip so far
            travelled, day, utility = find_duration(route, hotel, locations, attraction_dict, edges)
            
            # define the ending condition
            if places_remaining == [] or day >= stay:
                keep_running = False
            
            
            #### set absolute repeat count to 100
            counter = counter + 1
            if counter > w_counter:
                keep_running = False
            ####
            
#         unique_route = route
        
        unique_route = []
        for i in route:
            try:
                tmp = unique_route[-1]
            except:
                unique_route.append(i)
            
            if unique_route[-1] != i:
                unique_route.append(i)
        
        
        time_spent, days_spent, utility = find_duration(unique_route, hotel, locations, attraction_dict, edges)
        
        # day_visits = attractions_visited(route, attraction_dict)
        
        if run == 0:
            trip_duration = time_spent
            trip_day = days_spent
            trip_utility = utility
            # trip_visits = day_visits
        
        # handle a possible divisible by zero
        if float(utility) == 0.0:
            div_utility = 1
        else:
            div_utility = float(utility)
        
        if float(trip_utility) == 0.0:
            div_trip_utility = 1
        else:
            div_trip_utility = float(trip_utility)
        

            
        if time_spent*days_spent/float(div_utility) <= trip_duration*trip_day/float(div_trip_utility):
            trip_duration = time_spent
            trip_day = days_spent
            trip_utility = utility
            durations.append(trip_duration)
            paths.append(unique_route)
            days.append(trip_day)
            utilities.append(utility)
            keep_run.append(run)
            
    return paths, days, durations, utilities, keep_run

In [38]:
def show_path(paths, days, durations, utilities):
    for i in range(len(durations)):
        print 'Total travel time:', durations[i], 'minutes'
        print 'Days travelled:', days[i]
        print 'Score:', utilities[i]
        print 'Balanced Score:', durations[i] * days[i] / float(utilities[i])
        print 'Run:', keep_run[i]
        print paths[i]

In [39]:
# map the trip itinerary to the transportations

def trip_mapper(itinerary):
    trip = []

    for i in range(len(itinerary)-1):
        s_loc = itinerary[i]
        e_loc = itinerary[i+1]
        if s_loc != e_loc:
            trip.append(s_loc)
            s_coord = locations[s_loc]
            e_coord = locations[e_loc]
            key = [s_coord, e_coord]
            try:
                tmp_trip = edges[str(key)]['detail']['Choice 1']
                for i in tmp_trip:
                    if 'Choice ' not in i[0]:
                        trip.append(i)
            except:
                walk1 = ['Walk to next location']
                walk2 = ['$'+str(float(edges[str(key)]['result']['cost'][0])) , str(int(edges[str(key)]['result']['time'][0]))+' minutes']
                trip.append(walk1)
                trip.append(walk2)
            trip.append(e_loc)
            trip.append('--->')
    return trip

In [40]:
def expand_path(itinerary, hotel):
    expanded = []
    for k in range(len(itinerary)-1):
        i = itinerary[k]
        expanded.append(i)
        if i == hotel:
            if k != 0 and i != (len(itinerary)-1):
                expanded.append(hotel)
    expanded.append(hotel)
    reg_tail = True
    while reg_tail:
        if expanded[-1] == expanded[-2] and expanded[-1] == hotel:
            expanded = expanded[:len(expanded)-1]
        else:
            reg_tail = False
    return expanded

In [41]:
def route4plot(itinerary, attractions, locations):
    
    attraction_dict = change_df2dict(attractions)
    hotel = list(attractions[attractions.Duration == 'Hotel'].Locations)[0]
    itinerary = expand_path(itinerary, hotel)
    
    df_row = []
    day = 0
    stop = 0
    for k in range(len(itinerary)):
        
        i = itinerary[k]
        latitude = locations[i][0]
        longitude = locations[i][1]
        
        address = i
        
        if i == hotel and itinerary[k-1] == hotel:
            day += 1
            stop = 0
        
        try:
            s_loc = itinerary[k]
            e_loc = itinerary[k+1]
            if s_loc == e_loc:
                travel_time = 0
            else:
                a = locations[s_loc]
                b = locations[e_loc]
                key = [a,b]
                travel_time = edges[str(key)]['result']['time'][0]
        except:
            s_loc = itinerary[k]
            e_loc = hotel
            if s_loc == e_loc:
                travel_time = 0
            else:
                a = locations[s_loc]
                b = locations[e_loc]
                key = [a,b]
                travel_time = edges[str(key)]['result']['time'][0]
        
        try:
            travel_time = float(travel_time.replace(' minutes', ''))
        except:
            pass
        
        stop += 1
        
        try: 
            score = attraction_dict[i]['Score']
        except:
            score = 0
        
        try: 
            duration = attraction_dict[i]['Duration']
        except:
            duration = 0
        
        df_row.append([day, stop, latitude, longitude, address, travel_time, duration, score])
    df = pd.DataFrame(df_row)
    df.columns = ['day', 'stop', 'latitude', 'longitude', 'address', 'travel_time', 'duration', 'score']
    return df

In [42]:
def show_route_df(paths):
    count = 0
    route_df = pd.DataFrame()
    for i in paths:
        count += 1
        tmp_route_df = route4plot(i, attractions, locations)
        tmp_route_df['Iterations'] = count
        if count == 1:
            route_df = tmp_route_df
        else:
            route_df = pd.concat([route_df,tmp_route_df])
    route_df.to_csv('assets/datasets/plot_route.csv')
    return route_df

In [43]:
attractions = pd.read_excel('assets/datasets/ShortAttractions.xlsx')
attractions_list = list(attractions.Locations)
attraction_dict = change_df2dict(attractions)

In [44]:
def attractions_init():
    attractions = pd.read_excel('assets/datasets/ShortAttractions.xlsx')
    attractions_list = list(attractions.Locations)
    attraction_dict = change_df2dict(attractions)
    return attractions, attractions_list, attraction_dict

In [56]:
attractions, attractions_list, attraction_dict = attractions_init()
paths, days, durations, utilities, keep_run = simulation(attractions, stay = None, runs = 500)

100%|██████████| 500/500 [00:06<00:00, 71.76it/s]


In [57]:
show_route_df(paths)

,day,stop,latitude,longitude,address,travel_time,duration,score,Iterations
0,1,1,22.316837,114.169628,Sun Hing Building Kowloon,35.0,0,0,1
1,1,2,22.277194,114.179937,Morrison Building,20.0,30,100,1
2,1,3,22.267647,114.169170,Police Museum,80.0,120,50,1
3,1,4,22.377739,114.186262,Hong Kong Heritage Museum,55.0,180,11,1
4,1,5,22.324936,114.165056,Lui Seng Chun,10.0,50,17,1
5,1,6,22.309919,114.170696,Tin Hau Temple,25.0,60,14,1
6,1,7,22.282265,114.161528,Hong Kong City Hall,40.0,40,20,1
7,1,8,22.316837,114.169628,Sun Hing Building Kowloon,0.0,0,0,1
8,2,1,22.316837,114.169628,Sun Hing Building Kowloon,35.0,0,0,1
9,2,2,22.277763,114.161812,Hong Kong Park,5.0,120,50,1


In [58]:
show_path(paths, days, durations, utilities)

Total travel time: 1755.0 minutes
Days travelled: 3
Score: 358
Balanced Score: 14.7067039106
Run: 0
[u'Sun Hing Building Kowloon', u'Morrison Building', u'Police Museum', u'Hong Kong Heritage Museum', u'Lui Seng Chun', u'Tin Hau Temple', u'Hong Kong City Hall', u'Sun Hing Building Kowloon', u'Hong Kong Park', u'Peak Tram', u'Sam Tung Uk Museum', u'Sun Hing Building Kowloon', u'Cattle Depot Artist Village', u'King Yin Lei']
Total travel time: 1699.0 minutes
Days travelled: 3
Score: 358
Balanced Score: 14.2374301676
Run: 1
[u'Sun Hing Building Kowloon', u'Morrison Building', u'Peak Tram', u'Hong Kong Park', u'Police Museum', u'Sun Hing Building Kowloon', u'Tin Hau Temple', u'Cattle Depot Artist Village', u'Lui Seng Chun', u'Hong Kong City Hall', u'King Yin Lei', u'Sun Hing Building Kowloon', u'Sam Tung Uk Museum', u'Hong Kong Heritage Museum']
Total travel time: 1690.0 minutes
Days travelled: 3
Score: 358
Balanced Score: 14.1620111732
Run: 16
[u'Sun Hing Building Kowloon', u'Cattle Depot

In [54]:
itinerary = paths[-1]
trip = trip_mapper(paths[-1])
trip

[u'Sun Hing Building Kowloon',
 [u'KMB/NWFB: 104'],
 [u'SHANTUNG STREET MONG KOK/Shantung Street, Nathan Road'],
 [u'BANK OF CHINA TOWER/Bank of China Tower, Queensway'],
 [u'$9.3', u'35 minutes'],
 u'Hong Kong Park',
 '--->',
 u'Hong Kong Park',
 [u'TRAM: North Point - Whitty Street(Shek Tong Tsui)'],
 [u'Queensway (Murray Road)'],
 [u'Hennessy Road (Tonnochy Road)'],
 [u'$2.3', u'15 minutes'],
 u'Morrison Building',
 '--->',
 u'Morrison Building',
 [u'NWFB: 15B'],
 [u'Happy Valley Racecourse, Morrison Hill Road'],
 [u'Evergreen Villa, Stubbs Road'],
 [u'$9.6', u'10 minutes'],
 u'King Yin Lei',
 '--->',
 u'King Yin Lei',
 [u'NWFB: 15B'],
 [u'Evergreen Villa, Stubbs Road'],
 [u'Wan Chai Gap Road, Peak Road'],
 [u'$9.6', u'10 minutes'],
 u'Police Museum',
 '--->',
 u'Police Museum',
 [u'NWFB: 15'],
 [u'Wan Chai Gap Road, Stubbs Road'],
 [u'Pacific Place, Queensway'],
 [u'KMB/NWFB: 104'],
 [u'ADMIRALTY RAILWAY STATION/Admiralty Station, Queensway'],
 [u'NELSON STREET MONG KOK/Nelson Stre